In [10]:
%%bash
#To start, you'll need a file containing the geometry of ground state (aka, at the Franck-Condon point). This is 
#to be our starting geometry.
cat base_geometries/original_cartesian_coordinates/fc.xyz

10

C        -1.71206       -0.32706        0.11160
C        -0.30557       -0.48176        0.35394
C         0.75752       -0.01615       -0.49967
C         2.05157       -0.10807       -0.17313
H        -2.06407        0.20018       -0.76653
H        -2.42291       -0.66969        0.85372
H        -0.03213       -0.98685        1.27816
H         0.47549        0.43131       -1.44989
H         2.38131       -0.53852        0.76687
H         2.81691        0.26094       -0.84850


In [7]:
#Here's a gif of the molecule's FC geometry
from IPython.display import HTML
HTML('<img src="base_geometries/original_cartesian_coordinates/fc.gif">')

In [11]:
%%bash
#We'll also need a target geometry. For this demo, we'll use the transoid conical intersection.
cat base_geometries/original_cartesian_coordinates/Transoid.xyz

10

C    0.0099793130   -0.0426199469    0.0334010820
C    1.4861157025   -0.0976854547    0.0762123373
C    2.2613530886    1.0972557235   -0.1077587602
C    2.8086804038    0.9763007590    1.2397385526
H   -0.4698842455    0.3640063373   -0.8342116499
H   -0.5810457210   -0.2205813910    0.9080009880
H    1.9848176513   -1.0325999267    0.2159134468
H    1.8101598372    2.0061670885   -0.4459653405
H    3.5192864494    0.2117836737    1.4794448263
H    2.4257596824    1.5802733808    2.0391556726


In [12]:
#Here's a gif of the molecule's transoid geometry
from IPython.display import HTML
HTML('<img src="base_geometries/original_cartesian_coordinates/Transoid.gif">')

The purpose of the interpolator being demonstrated in this notebook is to make intermediate geometries between the starting and target geometries. It achieves this by calculating all the necessary bond lengths and angles, and adjusting them in a series of equally spaced steps. There is no calculation of interatomic forces or any other quantum mechanics-driven processes; it's simply a a series of equally spaced changes in bond lengths and angles. This allows us to make an energy profile of this transition with a comptational chemistry process such as Hartree-Fock, so we can calculate the energetic barrier.

Before the iterations can be made, the two geometries must be converted into internal coordinates (a.k.a. a z-matrix), which are similar to spherical coordinates. Let's convert the starting geometry:

In [13]:
%%bash
python z_matrix_maker.py base_geometries/original_cartesian_coordinates/fc.xyz
#This creates a z-matrix for the starting geometry and places it in the results folder.
cat results/fc.z

10
fc
C   
C 1 1.4355747579628164   
C 2 1.4406953363914246 1 126.02257190901  
C 3 1.3377754148211873 2 123.15814650033929 1 -175.15044892824105  
H 1 1.0830537173196906 2 120.51478314579748 3 1.5635149724705946  
H 1 1.0832567257118693 2 119.55027241625876 3 176.5538262209224  
H 2 1.0881488639427972 3 117.89328850511798 1 179.07511540782772  
H 3 1.0875106532351762 2 117.342676102004 4 179.81732309412024  
H 4 1.08518001737039 3 122.18244855414952 2 0.3502210410905301  
H 4 1.0853747429344394 3 120.4539422915961 2 179.75410180088488  


 [Wikipedia has a good in depth explanation of the z-matrix format](https://en.wikipedia.org/wiki/Z-matrix_(chemistry)). It is quite similar to normal spherical r, &theta;, &phi; coordinates. 
A shorter explanation: the first column are bond lengths (a line, so between 2 atoms), the second column are bond angles (between 3 atoms), and the third column are dihedral angles between two different planes made by 4 atoms (each plane shares two points; so for example, the angle between plane made by atoms A, B and C, and the plane made by atoms B, C, and D). You may notice the 1st, 2nd, and 3rd atom have incomplete rows; this is because these atoms are used to generate the coordinate system itself. The first atom establishes the origin, so it needs no data at all. The second atom establishes the X axis, so it needs one datum; a length (which becomes a unit vector). The third atom establishes the Y axis, so it needs a length and an angle to the previous vector. After that, all the atoms need three data points to establish their length, polar, and azimuthal angles (in z-matrix format, these are bond lengths, bond angles, and dihedral angles respectively).

The calculations of the lengths and angles themselves are fairly trivial if you are familiar with trigonometry/basic vector algebra, so I won't write an explanation here.

In [14]:
%%bash
#Let's also do this for our target geometry
python z_matrix_maker.py base_geometries/original_cartesian_coordinates/Transoid.xyz
cat results/Transoid.z

10
Transoid
C   
C 1 1.4777833583227715   
C 2 1.4362182246721835 1 120.29564863536793  
C 3 1.4594335549809925 2 90.87130612858174 1 -117.36025226594555  
H 1 1.0716183193346236 2 119.00515688767904 3 -55.99590569827763  
H 1 1.0704699257690835 2 121.4422039841109 3 112.15880495459534  
H 2 1.0687773873391442 3 119.51661583278056 1 -178.2181103776497  
H 3 1.069616244497664 2 121.31953530334093 4 128.22015245392652  
H 4 1.0709371706174067 3 120.97541387529039 2 -71.48145498938116  
H 4 1.0726038124631838 3 120.50333290460243 2 101.5576738530574  


Comparing the two, you can confirm what the two above visuals show: The ground state geometry has all the carbons almost planar (all the dihedral angles are near 0, or near 180 which is the same thing), but in the target geometry, one carbon has twisted out of the plane by about 60&deg;.

In [15]:
%%bash
#Now let's move those to the correct place, so the iterator knows where to find them later.
mv results/fc.z base_geometries/fc.z
mv results/Transoid.z base_geometries/Transoid.z

The iterator will rotate the angles from the starting geometry to the final geometry. Note, however, that since we want physically meaningful results, it doesn't make sense to rotate "the long way". Since low-energy processes are more physically likely to occur, an atom is more likely to rotate 10&deg; counterclockwise than it is to rotate 350&deg; clockwise. In the above geometries, there is one of these cases: See the dihedral angle for third Hydrogen (line 9) is going from roughly 179.1&deg; to -178.2&deg;. That's a gap of about 358 degrees, but really, you could just rotate 2 degrees the other way instead. This is achieved in the angle_fixer.py script which detects these cases and will add or subtract a full revolution from the angles so you are never going the "long way".

In [16]:
%%bash
python angle_fixer.py Transoid.z
cat base_geometries/fc.z
cat base_geometries/Transoid.z

Fixing angles for Transoid.z ...
10
fc
C   
C 1 1.4355747579628164   
C 2 1.4406953363914246 1 126.02257190901  
C 3 1.3377754148211873 2 123.15814650033929 1 -175.15044892824105  
H 1 1.0830537173196906 2 120.51478314579748 3 1.5635149724705946  
H 1 1.0832567257118693 2 119.55027241625876 3 176.5538262209224  
H 2 1.0881488639427972 3 117.89328850511798 1  -180.924884592
H 3 1.0875106532351762 2 117.342676102004 4 179.81732309412024  
H 4 1.08518001737039 3 122.18244855414952 2 0.3502210410905301  
H 4 1.0853747429344394 3 120.4539422915961 2 179.75410180088488  
10
Transoid
C   
C 1 1.4777833583227715   
C 2 1.4362182246721835 1 120.29564863536793  
C 3 1.4594335549809925 2 90.87130612858174 1 -117.36025226594555  
H 1 1.0716183193346236 2 119.00515688767904 3 -55.99590569827763  
H 1 1.0704699257690835 2 121.4422039841109 3 112.15880495459534  
H 2 1.0687773873391442 3 119.51661583278056 1 -178.2181103776497  
H 3 1.069616244497664 2 121.31953530334093 4 128.22015245392652  
H 4 1.

Note that dihedral hydrogen angle we were talking about before has changed to about -180.92&deg;, so a mot

In [17]:
%%bash
#Now we can run the iterator itself
python iteration_maker.py Transoid.z

Iterating Transoid.z...


100% (101 of 101) |#######################| Elapsed Time: 0:00:00 Time:  0:00:00


The iteratior has placed our interpolated frames in iterations/Transoid. They're still in the z-matrix format, however. To do useful analysis, we convert them back to xyz format.

In [18]:
%%bash
python mass_z_to_xyz.py Transoid.z

Converting Transoid frames from z to xyz...


100% (101 of 101) |#######################| Elapsed Time: 0:00:00 Time:  0:00:00


Now the interpolated frames are in a usable xyz format, in the same location where the z-matrices were stored. We can concatenate them together for an animation using a chemistry tool such as Avogadro.

In [19]:
%%bash
for frame in {0..100}
do
cat iterations/Transoid/$frame.xyz >> final_animations/Transoid_all_frames.xyz
done

This is the last step in the normal process (which is contained in butadiene_iterator.sh for convenience). A gif of the final animation can be seen below.

In [20]:
from IPython.display import HTML
HTML('<img src="base_geometries/original_cartesian_coordinates/transoid_transition_frames.gif">')